# Example 7: Worst case statistics

This example follows from previous ones. If something in the preamble is confusing, it is explained there.

This example focuses on the `wc_statistics!` used in the `WC` optimisation type of `Portfolio`.

## 7.1 Downloading the data

In [1]:
# using Pkg
# Pkg.add.(["StatsPlots", "GraphRecipes", "YFinance", "Clarabel", "HiGHS", "CovarianceEstimation", "SparseArrays"])
using Clarabel, CovarianceEstimation, DataFrames, Dates, GraphRecipes, HiGHS, YFinance,
      PortfolioOptimiser, Statistics, StatsBase, StatsPlots, TimeSeries, LinearAlgebra,
      PrettyTables, Random

fmt1 = (v, i, j) -> begin
    if j == 1
        return v
    else
        return if isa(v, Number)
            "$(round(v*100, digits=3)) %"
        else
            v
        end
    end
end;

function stock_price_to_time_array(x)
    coln = collect(keys(x))[3:end] # only get the keys that are not ticker or datetime
    m = hcat([x[k] for k ∈ coln]...) #Convert the dictionary into a matrix
    return TimeArray(x["timestamp"], m, Symbol.(coln), x["ticker"])
end
assets = ["AAL", "AAPL", "AMC", "BB", "BBY", "DELL", "DG", "DRS", "GME", "INTC", "LULU",
          "MARA", "MCI", "MSFT", "NKLA", "NVAX", "NVDA", "PARA", "PLNT", "SAVE", "SBUX",
          "SIRI", "STX", "TLRY", "TSLA"]
Date_0 = "2019-01-01"
Date_1 = "2023-01-01"
prices = get_prices.(assets; startdt = Date_0, enddt = Date_1)
prices = stock_price_to_time_array.(prices)
prices = hcat(prices...)
cidx = colnames(prices)[occursin.(r"adj", string.(colnames(prices)))]
prices = prices[cidx]
TimeSeries.rename!(prices, Symbol.(assets));

## 7.2 Instantiating an instance of `Portfolio`.

We'll compute basic statistics for this.

In [2]:
portfolio = Portfolio(; prices = prices,
                      # Continuous optimiser.
                      solvers = Dict(:Clarabel => Dict(:solver => Clarabel.Optimizer,
                                                       :check_sol => (allow_local = true,
                                                                      allow_almost = true),
                                                       :params => Dict("verbose" => false))),
                      # MIP optimiser for the discrete allocation.
                      alloc_solvers = Dict(:HiGHS => Dict(:solver => HiGHS.Optimizer,
                                                          :check_sol => (allow_local = true,
                                                                         allow_almost = true),
                                                          :params => Dict("log_to_console" => false))));

asset_statistics!(portfolio)

## 7.3 Effect of the Worst Case Mean Variance statistics

The previous tutorial showed how to perform worst case mean variance optimisations. This one goes into more detail on computing the uncertainty sets needed for this optimisation type.

The function in charge of doing so is `wc_statistics!` via the `WCType` type. Consult the docs for details.

There are a lot of combinations for this, so we will not be showing an exhaustive list. We will explore a representative subset. Since we used the default values for our previous tutorial we will explore a few of the other options.

We'll first use the default statistics for computing the optimised worst case mean variance portfolio.

In [3]:
# Set random seed for reproducible results.
Random.seed!(123)
wc_statistics!(portfolio)

We'll use the box set for the expected returns vector and the elliptical set for the covariance matrix. We'll maximise the risk-adjusted return ratio.

In [4]:
type = WC(; mu = Box(), cov = Ellipse())
obj = Sharpe(3.5 / 100 / 252)
w1 = optimise!(portfolio; type = type, obj = obj);

`WCType` can produce a wealth of uncertainty sets depending on the user provided parameters. You can experiment by changing the values of `wc` and computing the statistics again.

We'll now use a completely different set of parameters for computing the worst case statistics, but we will optimise the same problem.

In [5]:
wc = WCType(; cov_type = PortCovCor(; ce = CorGerber1(; normalise = true)),
            mu_type = MuBOP(), box = NormalWC(), ellipse = ArchWC(), k_sigma = KNormalWC(),
            k_mu = KGeneralWC(), diagonal = false)
wc_statistics!(portfolio, wc)
w2 = optimise!(portfolio; type = type, obj = obj)

pretty_table(DataFrame(; tickers = w1.tickers, w1 = w1.weights, w2 = w2.weights);
             formatters = fmt1)

┌─────────┬──────────┬──────────┐
│ tickers │       w1 │       w2 │
│  String │  Float64 │  Float64 │
├─────────┼──────────┼──────────┤
│     AAL │  0.053 % │  0.027 % │
│    AAPL │ 89.981 % │ 93.969 % │
│     AMC │   0.04 % │  0.021 % │
│      BB │  0.057 % │  0.029 % │
│     BBY │  0.193 % │  0.087 % │
│    DELL │   0.21 % │  0.094 % │
│      DG │  1.087 % │  0.356 % │
│     DRS │  0.759 % │  0.283 % │
│     GME │  0.632 % │  0.227 % │
│    INTC │  0.083 % │   0.04 % │
│    LULU │  0.395 % │  0.171 % │
│    MARA │  0.161 % │  0.075 % │
│     MCI │  0.182 % │  0.082 % │
│    MSFT │  0.708 % │  0.276 % │
│    NKLA │   0.04 % │  0.021 % │
│    NVAX │  0.051 % │  0.027 % │
│    NVDA │   1.15 % │  0.394 % │
│    PARA │  0.058 % │  0.029 % │
│    PLNT │  0.136 % │  0.064 % │
│    SAVE │   0.05 % │  0.026 % │
│    SBUX │  0.255 % │  0.112 % │
│    SIRI │  0.151 % │  0.068 % │
│     STX │  0.194 % │  0.087 % │
│    TLRY │  0.025 % │  0.014 % │
│    TSLA │  3.349 % │  3.423 % │
└─────────┴───

When compared to the previous tutorial, the takeaway here is that the type of uncertainty set used has much more of an impact on the results of the optimisation 6.4 Optimising the portfolio than the parameters used to compute the worst case sets. However, more robust statistics will produce more robust uncertainty sets.

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*